# Final ML Project
## Daniel Bernal, Raymond Vuong, Rohit Punjani, and Neal Davar 


In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor





training = pd.read_csv("train.csv")
testing = pd.read_csv("housing_testing.csv")

IL_F_ITERATIONS = 50

print(len(training))

# 1. Data Cleaning:

training.drop('Id', axis=1, inplace=True)
# fillna on the basis of whether MasVnrArea, LotFrontage, or any other categorical feature with na values
training['MasVnrArea'].fillna(0, inplace=True)
training['LotFrontage'].fillna(0, inplace=True)
training['GarageYrBlt'].fillna(training['GarageYrBlt'].mean(), inplace=True)
training.fillna('None', inplace=True)

# drop duplicates:
training.drop_duplicates(keep=False, inplace=True)

# one hot encoding categorical variables:
training = pd.get_dummies(training, columns=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'], drop_first=True)


# use isolation forests to find potential outliers:
freq_outlier_map = {} 
for i in range(0, IL_F_ITERATIONS):
    anomalies = IsolationForest().fit_predict(training, 0.5)
    training['anomalies'] = anomalies
    outlier_indices = training.loc[training['anomalies'] == -1].index
    # add outlier freqs to map
    for j in range(0, len(outlier_indices)):
        count = 0
        if outlier_indices[j] in freq_outlier_map: 
            count = freq_outlier_map[outlier_indices[j]]
        freq_outlier_map[outlier_indices[j]] = count + 1
    
    inlier_indices = training.loc[training['anomalies'] == 1].index

    
# drop outliers that are detected as anomalies more than 10% of the time
final_outlier_indices = []
print('Total # of Outliers: ')
for outlier_index in freq_outlier_map.keys(): 
    if freq_outlier_map[outlier_index] > (0.10 * IL_F_ITERATIONS):
        final_outlier_indices.append(outlier_index)
print(len(final_outlier_indices))
print('Number of data points before outlier removal: ')
print(len(training))
training.drop(index=final_outlier_indices, inplace=True)
print('Number of data points before after outlier removal: ')
print(len(training))


# Anomaly Detection post PCA:


# 2. Data Exploration: 



# 3. Feature Engineering:


# consolidate bathroom columns into one col:
training['TotalBathrooms'] = \
        training['FullBath'] + (0.5 * training['HalfBath']) + \
        training['BsmtFullBath'] + (0.5 * training['BsmtHalfBath'])

print('b')        

training.drop(['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'], axis=1)

# consolidate porch area into one column
training['TotalPorchSF'] = \
        training['OpenPorchSF'] + training['EnclosedPorch'] + \
        training['3SsnPorch'] + training['ScreenPorch']
    
training.drop(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', '3SsnPorch'], axis=1)




# TODO: scale the data 
def normalize_data(train, test):
    # your code goes here
    train_norm = (train - train.min()) / (train.max() - train.min())
    test_norm = (test - test.min()) / (test.max() - test.min())
    return train_norm, test_norm

train_norm, test_norm = normalize_data(training, testing)


# TODO: use PCA to reduce overall dimensionality

# TODO: SMOTE for regression to manage class imbalances



# 3. Modeling: 4 different type of regression models with cross validation:

# - Model I: Linear Regression

# - Model II: Regression Tree (Nonlinear Regression)

# - Model III: SVM Regression





labels = train_norm['SalePrice']
labels.values.ravel()
features = train_norm.drop('SalePrice', axis = 1)
print(features.shape)
print(labels.shape)
print(features.head())
print(labels.head())


linReg = LinearRegression()
scores = cross_val_score(linReg, features, labels, cv=10)
print("Accuracy:", scores.mean()*100)








1460
Total # of Outliers: 
21
Number of data points before outlier removal: 
1460
Number of data points before after outlier removal: 
1439
b


/var/folders/wg/fd60dmln4nq615ms_jvbcyj40000gn/T/ipykernel_27248/666622852.py:104: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_norm = (test - test.min()) / (test.max() - test.min())


TypeError: unsupported operand type(s) for -: 'str' and 'str'